In [1]:
import T4sigWGAN as T4
from torch.utils.data import DataLoader, random_split

In [2]:
import argparse
import torch

parser = argparse.ArgumentParser()

# initial setting
parser.add_argument("--seed", default=42, type=int)
parser.add_argument("--gpu_id", type=str, default="0", help="gpu_id")
parser.add_argument("--no_cuda", action="store_true")
parser.add_argument("--batch_size", type=int, default=32, help="number of batch_size")
parser.add_argument("--epochs", type=int, default=1, help="number of epochs")
parser.add_argument("--data_dir", default="../Database/", type=str)
parser.add_argument("--output_dir", default="output/", type=str)
parser.add_argument("--device", default=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

# log location
parser.add_argument("--model_name", default="DenseNet", type=str)
parser.add_argument("--data_name", default="IMAGENET", type=str)

# model parameter
parser.add_argument("--input_size", type=int, default=2, help="hidden size of transformer model")
parser.add_argument("--hidden_size", type=int, default=512, help="hidden size of transformer model")
parser.add_argument("--num_layers", type=int, default=2, help="number of layers")
parser.add_argument("--nonlinearity", default="tanh", type=str)
parser.add_argument("--bias", default=True, type=bool)
parser.add_argument("--batch_first", default=True, type=bool)

# trainer parameter
parser.add_argument("--adam_beta1", type=float, default=0.9, help="adam first beta value")
parser.add_argument("--adam_beta2", type=float, default=0.999, help="adam second beta value")
parser.add_argument("--lr", type=float, default=0.001, help="learning rate of adam")
parser.add_argument("--weight_decay", type=float, default=0.0, help="weight_decay of adam")
parser.add_argument("--log_freq", type=int, default=5, help="per epoch print res")

# main
parser.add_argument("--using_pretrain", default=False, action="store_true")

args, unknown = parser.parse_known_args()

In [3]:
## decoder parameters
decoder_config = {
    'ts_shape': [30,256],
    'num_heads': 2,
    'k_size': 4,
    'dilations': [1, 4],
    'dropout': 0.2,
}

In [4]:
class Configs:
    def __init__(self,):
        self.enc_in = 256
        self.dims = [256, 128, 256]
        self.large_size = [5, 5, 3]
        self.small_size = [5, 3, 3]
        self.small_kernel_merged = False
        self.dropout = 0.1
        self.head_dropout = 0.2
        self.revin = True
        self.affine = True
        self.decomposition = True
        self.kernel_size = 25

In [5]:
## supervisor parameters
supervisor_config = Configs()

In [6]:
## generator parameters
encoder_config = {
    "input_dim": 5,
    "hidden_dim": 256,
    "augmentations": [
        {"name": "LeadLag"},
    ],
    "depth": 2,
    "output_dim": 10,
    "len_interval_u": 50,
    "init_fixed": True,
    "batch_size": 128,
    "window_size": 30,
    "device": "cuda:0",
}

if encoder_config.get('augmentations') is not None:
    encoder_config['augmentations'] = T4.parse_augmentations(encoder_config.get('augmentations'))

In [7]:
## generator parameters
logsig_config = {
    "input_dim": 5,
    "hidden_dim": 256,
    "augmentations": [
        {"name": "LeadLag"},
    ],
    "depth": 2,
    "output_dim": 10,
    "len_noise": 1000,
    "len_interval_u": 50,
    "init_fixed": True,
    "batch_size": 128,
    "window_size": 30,
    "device": "cuda:0",
}

if logsig_config.get('augmentations') is not None:
    logsig_config['augmentations'] = T4.parse_augmentations(logsig_config.get('augmentations'))

In [8]:
## discriminator parameters
discriminator_config = {
    'n_epochs': 3000,
    'batch_size': 1000,
    'lr_D': 1e-7,
    'lr_G': 1e-6,
    'temp': 0.01,
    'b1': 0.5,
    'b2': 0.999,
    'latent_dim': 1000,
    'n_len': 50000,
    'n_rows': 5,
    'n_cols': 100,
    'n_critic_G': 1,
    'n_critic_D': 1,
    'static_way': 'LShort',
    'strategies': ['Port', 'MR', 'TF'],
    'n_trans': 50,
    'Cap': 10,
    'WH': 10,
    'ratios': [1.0, 1.0],
    'thresholds_pct': [[31, 69]],
    'data_name': '1_Gauss+1_AR50+1_AR-12+1_GARCH-T5+1_GARCH-T10',
    'tickers': ['Gauss', 'AR50', 'AR-12', 'GARCH-T5', 'GARCH-T10'],
    'noise_name': 't5',
    'alphas': [0.05],
    'W': 10.0,
    'score': 'quant',
    'numNN': 10,
    'project': True,
    'version': 'Test1'
}

In [9]:
## trainer parameters
sigwgan_config = {
    "batch_size": 256,
    "epoch": 2500,  # 2000 is enough
    "lr_generator": 5e-4,
    "augmentations": [
        {"name": "AddTime"},
        {"name": "LeadLag"},
        {"name": "VisiTrans", "type": "I"}
    ],
    "device": "cuda:0",
    "depth": 2
}

In [10]:
Encoder = T4.LogSigRNNDecoder(**encoder_config).to(args.device)
Encoder

LogSigRNNDecoder(
  (rnn): Sequential(
    (0): FeedForwardNN(
      (network): Sequential(
        (0): Linear(in_features=311, out_features=256, bias=True)
        (1): PReLU(num_parameters=1)
        (2): Linear(in_features=256, out_features=256, bias=True)
        (3): PReLU(num_parameters=1)
        (4): Linear(in_features=256, out_features=256, bias=True)
      )
    )
    (1): Tanh()
  )
  (linear): Linear(in_features=256, out_features=256, bias=False)
  (initial_nn): Sequential(
    (0): ResidualNN(
      (network): Sequential(
        (0): ResidualBlock(
          (activation): ReLU()
          (linear): Linear(in_features=5, out_features=256, bias=True)
        )
        (1): ResidualBlock(
          (activation): ReLU()
          (linear): Linear(in_features=256, out_features=256, bias=True)
        )
        (2): Linear(in_features=256, out_features=256, bias=True)
      )
    )
    (1): Tanh()
  )
)

In [11]:
Decoder = T4.TimesFormerDecoder(**decoder_config).to(args.device)
Decoder

TimesFormerDecoder(
  (timesformer_blocks): ModuleList(
    (0): TimesFormerLayer(
      (conv): Conv1d(256, 256, kernel_size=(4,), stride=(1,), padding=same)
      (relu): ReLU()
      (norm_tcn): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.2, inplace=False)
      (mha1): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (ff_conv): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
      (ff_relu): ReLU()
      (ff_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (cross1): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (cross2): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (norm2): LayerNorm((256,), eps=1e-05, elem

In [12]:
Supervisor =T4.ModernTCN(supervisor_config).to(args.device)
Supervisor

ModernTCN(
  (revin): RevIN()
  (decomp): series_decomp(
    (moving_avg): moving_avg(
      (avg): AvgPool1d(kernel_size=(25,), stride=(1,), padding=(0,))
    )
  )
  (conv_layers): ModuleList(
    (0): ReparamLargeKernelConv(
      (lkb_origin): Sequential(
        (conv): Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (small_conv): Sequential(
        (conv): Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): ReparamLargeKernelConv(
      (lkb_origin): Sequential(
        (conv): Conv1d(256, 128, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (small_conv): Sequential(
        (conv): Conv1d(256

In [13]:
Generator = T4.LogSigRNNGenerator(**logsig_config).to(args.device)
Generator

LogSigRNNGenerator(
  (rnn): Sequential(
    (0): FeedForwardNN(
      (network): Sequential(
        (0): Linear(in_features=311, out_features=256, bias=True)
        (1): PReLU(num_parameters=1)
        (2): Linear(in_features=256, out_features=256, bias=True)
        (3): PReLU(num_parameters=1)
        (4): Linear(in_features=256, out_features=256, bias=True)
      )
    )
    (1): Tanh()
  )
  (linear): Linear(in_features=256, out_features=256, bias=False)
  (initial_nn): Sequential(
    (0): ResidualNN(
      (network): Sequential(
        (0): ResidualBlock(
          (activation): ReLU()
          (linear): Linear(in_features=5, out_features=256, bias=True)
        )
        (1): ResidualBlock(
          (activation): ReLU()
          (linear): Linear(in_features=256, out_features=256, bias=True)
        )
        (2): Linear(in_features=256, out_features=256, bias=True)
      )
    )
    (1): Tanh()
  )
)

In [14]:
Discriminator = T4.Discriminator(discriminator_config).to(args.device)
Discriminator

Discriminator(
  (model): Sequential(
    (0): Linear(in_features=1000, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Linear(in_features=128, out_features=2, bias=True)
  )
)

In [15]:
model = T4.TimeGAN(Encoder, Decoder, Generator, Supervisor, Discriminator).to(args.device)

In [16]:
# # Run TimeGAN
# generated_data = T4.train_T4sigWGAN(model, x_real_rolled, x_real_rolled.shape[1], sigwgan_config)
# print('Finish Synthetic Data Generation')

In [17]:
train_dataset = T4.StockTimeSeriesDataset()

train_size = int(0.9 * len(train_dataset))  # 80% for training
val_size = len(train_dataset) - train_size  # 20% for validation
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# DataLoader 설정
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=2)

In [ ]:


# save model
# checkpoint = args_str + ".pt"
# args.checkpoint_path = os.path.join(args.output_dir, checkpoint)

# trainer
trainer = T4.ERTrainer(
    model, train_dataloader, val_dataloader, None, None, args
)

# model training
for epoch in range(args.epochs):
    trainer.train(epoch)
    val_loss = trainer.valid(epoch)

# train log image save
trainer.evaluate()

Total Parameters: 9076914


In [ ]:
# trainer
trainer = T4.GSTrainer(
    model, train_dataloader, val_dataloader, None, None, args
)

# model training
for epoch in range(args.epochs):
    trainer.train(epoch)
    val_loss = trainer.valid(epoch)

# train log image save
trainer.evaluate()

In [40]:
# trainer
trainer = T4.FinetuneTrainer(
    model, train_dataloader, val_dataloader, None, None, args
)

# model training
for epoch in range(args.epochs):
    trainer.train(epoch)
    val_loss = trainer.valid(epoch)

# train log image save
trainer.evaluate()

Total Parameters: 9076914


DenseNet EP_train:0:   0%|| 0/26 [11:15<?, ?it/s]

KeyboardInterrupt



In [ ]:
metric_iteration = 5

discriminative_score = list()
for _ in range(metric_iteration):
    temp_disc = T4.discriminative_score_metrics(ori_data, generated_data)
    discriminative_score.append(temp_disc)

print('Discriminative score: ' + str(np.round(np.mean(discriminative_score), 4)))

In [ ]:
predictive_score = list()
for tt in range(metric_iteration):
    temp_pred = T4.predictive_score_metrics(ori_data, generated_data)
    predictive_score.append(temp_pred)

print('Predictive score: ' + str(np.round(np.mean(predictive_score), 4)))

In [ ]:
T4.visualization(ori_data, generated_data, 'pca')
T4.visualization(ori_data, generated_data, 'tsne')